In [78]:
import pandas as pd
import json
from tqdm import tqdm
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer
import stopwordsiso as stopwords
import pickle
class Languages:
    esga = 'ES-GA'
    si = 'SI'
    gb = 'GB'
    hu = 'HU'
    ua = 'UA'
    all = [esga, si, gb, hu, ua]

In [18]:
sentences = {}
for language in Languages.all:
    print('loading', language)
    sentences[language] = pd.read_feather(f'artefacts/pandas_frames/{language}_parlamint_sentences.feather')
metadata = pd.read_feather('artefacts/pandas_frames/parlamint_metadata.feather')

In [3]:
metadata

,index,ID,Title,Date,Body,Term,Session,Meeting,Sitting,Agenda,Subcorpus,Speaker_role,Speaker_MP,Speaker_Minister,Speaker_party,Speaker_party_name,Party_status,Speaker_name,Speaker_gender,Speaker_birth
0,0,ParlaMint-ES-GA_2017-09-26-DSPG040.u1,Minutes of the Assembly of the Galician Parlia...,2017-09-26,Unicameralism,010,,040,2017-09-26,,Reference,Chairperson,MP,-,PPdeG,Partido Popular de Galicia,Coalition,"Santalices Vieira, Miguel Ángel",M,1955
1,1,ParlaMint-ES-GA_2017-09-26-DSPG040.u2,Minutes of the Assembly of the Galician Parlia...,2017-09-26,Unicameralism,010,,040,2017-09-26,,Reference,Chairperson,MP,-,PPdeG,Partido Popular de Galicia,Coalition,"Santalices Vieira, Miguel Ángel",M,1955
2,2,ParlaMint-ES-GA_2017-09-26-DSPG040.u3,Minutes of the Assembly of the Galician Parlia...,2017-09-26,Unicameralism,010,,040,2017-09-26,,Reference,Regular,MP,-,PPdeG,Partido Popular de Galicia,Coalition,"Arias Rodríguez, Raquel",F,1966
3,3,ParlaMint-ES-GA_2017-09-26-DSPG040.u4,Minutes of the Assembly of the Galician Parlia...,2017-09-26,Unicameralism,010,,040,2017-09-26,,Reference,Chairperson,MP,-,PPdeG,Partido Popular de Galicia,Coalition,"Santalices Vieira, Miguel Ángel",M,1955
4,4,ParlaMint-ES-GA_2017-09-26-DSPG040.u5,Minutes of the Assembly of the Galician Parlia...,2017-09-26,Unicameralism,010,,040,2017-09-26,,Reference,Chairperson,MP,-,PPdeG,Partido Popular de Galicia,Coalition,"Santalices Vieira, Miguel Ángel",M,1955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1365545,213,ParlaMint-UA_2019-11-13-m0.u214,"Ukrainian parliamentary corpus ParlaMint-UA, t...",2019-11-13,Unicameralism,9,2,,2019-11-13,,Reference,Regular,MP,-,фЄС,"Фракція політичної партії ""Європейська солідар...",Opposition,"Олексійович Гончаренко, Олексій",M,1980
1365546,214,ParlaMint-UA_2019-11-13-m0.u215,"Ukrainian parliamentary corpus ParlaMint-UA, t...",2019-11-13,Unicameralism,9,2,,2019-11-13,,Reference,Chairperson,MP,-,,,,"Олександрович Разумков, Дмитро",M,1983
1365547,215,ParlaMint-UA_2019-11-13-m0.u216,"Ukrainian parliamentary corpus ParlaMint-UA, t...",2019-11-13,Unicameralism,9,2,,2019-11-13,,Reference,Regular,MP,-,фЄС,"Фракція політичної партії ""Європейська солідар...",Opposition,"Володимирівна Геращенко, Ірина",F,1971
1365548,216,ParlaMint-UA_2019-11-13-m0.u217,"Ukrainian parliamentary corpus ParlaMint-UA, t...",2019-11-13,Unicameralism,9,2,,2019-11-13,,Reference,Chairperson,MP,-,,,,"Олександрович Разумков, Дмитро",M,1983


In [82]:
sentences[Languages.si]

,sentence_id,speech_id,language,text,lemmas
0,ParlaMint-SI_2014-12-23-SDZ7-Izredna-08.ana.se...,ParlaMint-SI_2014-12-23-SDZ7-Izredna-08.u1,SI,Spoštovane kolegice poslanke in kolegi poslanc...,spoštovan kolegica poslanka in kolega poslanec...
1,ParlaMint-SI_2014-12-23-SDZ7-Izredna-08.ana.se...,ParlaMint-SI_2014-12-23-SDZ7-Izredna-08.u1,SI,"Začenjam 8. izredno sejo Državnega zbora, ki s...",začenjati izreden seja državen zbor ki biti on...
2,ParlaMint-SI_2014-12-23-SDZ7-Izredna-08.ana.se...,ParlaMint-SI_2014-12-23-SDZ7-Izredna-08.u1,SI,"Obveščen sem, da se današnje seje ne more udel...",obveščen biti da se današnji seja ne moči udel...
3,ParlaMint-SI_2014-12-23-SDZ7-Izredna-08.ana.se...,ParlaMint-SI_2014-12-23-SDZ7-Izredna-08.u1,SI,Vse prisotne lepo pozdravljam.,ves prisoten lepo pozdravljati
4,ParlaMint-SI_2014-12-23-SDZ7-Izredna-08.ana.se...,ParlaMint-SI_2014-12-23-SDZ7-Izredna-08.u1,SI,Prehajamo na določitev dnevnega reda 8. izredn...,prehajati na določitev dneven red izreden seja...
...,...,...,...,...,...
3876178,ParlaMint-SI_2019-03-05-SDZ8-Redna-05.ana.seg8...,ParlaMint-SI_2019-03-05-SDZ8-Redna-05.u170,SI,S tem prekinjam to točko dnevnega reda.,z ta prekinjati ta točka dneven red
3876179,ParlaMint-SI_2019-03-05-SDZ8-Redna-05.ana.seg8...,ParlaMint-SI_2019-03-05-SDZ8-Redna-05.u170,SI,"Prekinjam pa tudi 5. sejo zbora, ki jo bomo na...",prekinjati pa tudi seja zbor ki on biti nadalj...
3876180,ParlaMint-SI_2019-03-05-SDZ8-Redna-05.ana.seg8...,ParlaMint-SI_2019-03-05-SDZ8-Redna-05.u170,SI,Lep večer še naprej vsem!,lep večer še naprej ves
3876181,ParlaMint-SI_2019-03-05-SDZ8-Redna-05.ana.seg8...,ParlaMint-SI_2019-03-05-SDZ8-Redna-05.u170,SI,( SEJA JE BILA PREKINJENA 5.,seja biti biti prekinjen


In [36]:
for language in tqdm(Languages.all):
    sentences[language].speech_id = sentences[language].speech_id.apply(lambda x: x.replace('.ana', ''))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:02<00:00,  2.27it/s]


In [43]:
speeches = {}
for language in tqdm(sentences):
    with open(f'artefacts/pandas_frames/speech2lemmas_{language}.pkl', 'rb') as f:
        s2l = pickle.load(f)
        for s in list(s2l.keys()):
            s2l[s.replace('.ana', '')] = s2l[s]
            s2l.pop(s)
        speeches[language] = s2l

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:18<00:00,  3.74s/it]


## Prepare data for TF-IDF

### Time period
Based on Bojan's sentiment scores

In [7]:
sentences_gb = sentences[Languages.gb][sentences[Languages.gb].speech_id.str.contains('commons')]
metadata_gb = metadata[metadata.ID.isin(sentences_gb.speech_id.unique())]

In [9]:
may_1_to_32_2020_metadata = metadata_gb[(metadata_gb.Date >= '2020-05-01') & (metadata_gb.Date <= '2020-05-31')]
may_1_to_32_2020_sentences = sentences_gb[sentences_gb.speech_id.isin(may_1_to_32_2020_metadata.ID)]

In [10]:
con_sentences = may_1_to_32_2020_sentences[may_1_to_32_2020_sentences.speech_id.isin(
    may_1_to_32_2020_metadata[may_1_to_32_2020_metadata.Speaker_party == 'CON'].ID.unique())]
lab_sentences = may_1_to_32_2020_sentences[may_1_to_32_2020_sentences.speech_id.isin(
    may_1_to_32_2020_metadata[may_1_to_32_2020_metadata.Speaker_party == 'LAB'].ID.unique())]
'con:', len(con_sentences), 'lab:', len(lab_sentences)

('con:', 10021, 'lab:', 4672)

In [11]:
# get lemma groups by speach_id
lemmas = [
    ('con', ' '.join(con_sentences.groupby('speech_id').lemmas.apply(lambda x: ' '.join(x)))),
    ('lab', ' '.join(lab_sentences.groupby('speech_id').lemmas.apply(lambda x: ' '.join(x)))),
]

### By list of speeches

In [67]:
lemmas = []
for speaker_name, speech_ids in tqdm(metadata[metadata.language == Languages.si].groupby('Speaker_name').ID):
    lemmas.append((
        speaker_name,
        ' '.join([speeches[Languages.si][sid] for sid in speech_ids.values if sid in speeches[Languages.si]])
    ))
lemmas = list(sorted(lemmas, key=lambda x: len(x[1])))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 973/973 [00:00<00:00, 2372.28it/s]


#### Limit speakers

In [73]:
lemmas = []
speakers = ['Janša, Ivan Janez', 'Janković, Zoran']
for speaker_name, speech_ids in tqdm(metadata[(metadata.language == Languages.si) & (metadata.isin(speakers).Speaker_name)].groupby('Speaker_name').ID):
    lemmas.append((
        speaker_name,
        ' '.join([speeches[Languages.si][sid] for sid in speech_ids.values if sid in speeches[Languages.si]])
    ))
lemmas = list(sorted(lemmas, key=lambda x: len(x[1])))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 483.08it/s]


### Topics

In [143]:
lemmas = {}
for topic in ['healthcare', 'eu', 'war']:
    print('processing', topic)
    topic_lemmas = []
    artefacts_by_topic = Path('artefacts/by_topic')
    topic_speeches = set(pd.read_feather(artefacts_by_topic / f'{topic}.feather').SI.values)
    speech2lemmas = sentences[Languages.si].groupby('speech_id').lemmas.apply(lambda x: ' '.join(x))

    for party, party_metadata in tqdm(metadata[metadata.language == Languages.si].groupby('Speaker_party')):
        party = party if party.strip() else '/'
        ids = [i for i in party_metadata.ID if i in speech2lemmas and i in topic_speeches]
        topic_lemmas.append(
            (party, ' '.join(speech2lemmas[ids]))
        )
    topic_lemmas = list(sorted(topic_lemmas, key=lambda x: len(x[1]), reverse=True))
    lemmas[topic] = topic_lemmas

processing healthcare


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:00<00:00, 59.09it/s]


processing eu


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:00<00:00, 57.18it/s]


processing war


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:00<00:00, 59.14it/s]


## Calculate TF-IDF

In [144]:
if type(lemmas) is list:
    lemmas['general'] = lemmas

tf_idf = {}
for topic, topic_lemmas in lemmas.items():
    print('processing', topic)
    sw = stopwords.stopwords('sl')
    vectorizer = TfidfVectorizer(stop_words=list(sw), min_df=6, max_df=0.7)
    documents = [l for _, l in topic_lemmas]
    X = vectorizer.fit_transform(documents)

    tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

    tfidf_df = tfidf_df.T
    top_words = {}
    for i in range(len(documents)):
        sorted_df = tfidf_df[i].sort_values(ascending=False)
        top_words[topic_lemmas[i][0]] = sorted_df.index.tolist()
        top_words[f'{topic_lemmas[i][0]}_scores'] = sorted_df.tolist()
    tf_idf[topic] = pd.DataFrame(top_words)

processing healthcare


/home/vidklopcic/anaconda3/envs/dhh23/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['etc', 'idr', 'npr', 'oz', 'prbl', 'pribl', 'tj'] not in stop_words.
  warnings.warn(


processing eu


/home/vidklopcic/anaconda3/envs/dhh23/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['etc', 'idr', 'npr', 'oz', 'prbl', 'pribl', 'tj'] not in stop_words.
  warnings.warn(


processing war


/home/vidklopcic/anaconda3/envs/dhh23/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['etc', 'idr', 'npr', 'oz', 'prbl', 'pribl', 'tj'] not in stop_words.
  warnings.warn(


In [145]:
tf_idf['eu']

,SDS,SDS_scores,SD,SD_scores,NSi,NSi_scores,LDS,LDS_scores,SMC,SMC_scores,...,SLS+SKD,SLS+SKD_scores,Lipa,Lipa_scores,ZaAB,ZaAB_scores,Konkretno,Konkretno_scores,DLGV,DLGV_scores
0,sd,0.137675,os,0.216228,krščanski,0.526912,lds,0.607226,smc,0.718560,...,sekretariat,0.507256,lipa,0.870299,pripravništvo,0.506061,vesoljski,0.462136,multipleks,0.957739
1,tožilec,0.134444,idrija,0.144202,epidemija,0.134421,azbest,0.129492,migrantski,0.182453,...,skd,0.304986,predsedovati,0.145718,konsolidacija,0.379225,valuten,0.244525,radiodifuzija,0.130383
2,bratušek,0.129051,tožilec,0.108097,migrantski,0.125160,protokol,0.111354,epidemija,0.128221,...,maloobmejen,0.286784,pomočnik,0.130742,volonterski,0.353905,pandemija,0.212566,izdajatelj,0.097787
3,epidemija,0.122518,srebro,0.107789,dutb,0.117522,prodajalna,0.088959,dutb,0.120788,...,sls,0.200583,psihiatričen,0.094785,dutb,0.231020,frank,0.162963,frekvenca,0.065191
4,alenka,0.120427,epidemija,0.107724,sib,0.095998,terminal,0.083361,nlb,0.104121,...,krma,0.196322,bahati,0.080736,pripravnik,0.198274,epidemija,0.162806,analogno,0.055104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11478,poizvedovanje,0.000000,enomesečen,0.000000,rubikon,0.000000,poduk,0.000000,političarka,0.000000,...,mors,0.000000,moto,0.000000,minuten,0.000000,minuten,0.000000,mikroorganizem,0.000000
11479,zaslužkarstvo,0.000000,und,0.000000,ruda,0.000000,uber,0.000000,sponzoriranje,0.000000,...,morski,0.000000,motor,0.000000,minutka,0.000000,minutka,0.000000,mikroplastika,0.000000
11480,reprezentativnost,0.000000,umrljivost,0.000000,intervencionizem,0.000000,podtaknjenec,0.000000,mesen,0.000000,...,moskva,0.000000,motorist,0.000000,miran,0.000000,miran,0.000000,mikropodjetje,0.000000
11481,reproduktiven,0.000000,enostavnost,0.000000,interpelacijski,0.000000,ubijanje,0.000000,bojišče,0.000000,...,moten,0.000000,mozaik,0.000000,miriti,0.000000,miriti,0.000000,miks,0.000000


In [146]:
tf_idf['healthcare']

,SDS,SDS_scores,SD,SD_scores,NSi,NSi_scores,LDS,LDS_scores,SMC,SMC_scores,...,SLS+SKD,SLS+SKD_scores,Lipa,Lipa_scores,Konkretno,Konkretno_scores,ZaAB,ZaAB_scores,DLGV,DLGV_scores
0,ukc,0.228982,covid,0.217554,krščanski,0.404438,azbest,0.289077,smc,0.616946,...,sekretariat,0.400341,lipa,0.806215,covid,0.444869,zavezništvo,0.475037,digitalen,0.502058
1,migrant,0.186628,azbest,0.215807,covid,0.173513,bručan,0.187050,lekarniški,0.224647,...,krma,0.245514,draginjski,0.223297,nalezljiv,0.250543,pripravništvo,0.448070,frekvenca,0.284492
2,smc,0.138609,azbesten,0.198717,zzzs,0.155795,azbesten,0.155008,vajeništvo,0.142609,...,sls,0.230563,rudnik,0.148194,pandemija,0.181412,volonterski,0.318559,izdajatelj,0.261520
3,covid,0.136605,okužba,0.158828,ukc,0.122627,vzajemen,0.145601,covid,0.139345,...,šmartno,0.170303,trbovlje,0.130010,cepljenje,0.174009,grad,0.262090,odpis,0.251029
4,tožilec,0.131687,idrija,0.135971,regresen,0.113254,keber,0.117093,migrant,0.117117,...,igralništvo,0.166518,drobitev,0.111817,pkp,0.162408,viseti,0.180187,janković,0.222866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11736,bentiti,0.000000,tipično,0.000000,neminovno,0.000000,pečečnik,0.000000,pad,0.000000,...,moralo,0.000000,moraliziranje,0.000000,moderno,0.000000,mladoletniški,0.000000,milica,0.000000
11737,pkpeji,0.000000,podučen,0.000000,tratiti,0.000000,petrinov,0.000000,iracionalen,0.000000,...,moratorij,0.000000,moralizirati,0.000000,modificiran,0.000000,mladost,0.000000,miligram,0.000000
11738,mobilizirati,0.000000,thc,0.000000,travmatičen,0.000000,petišovci,0.000000,sposoditi,0.000000,...,moravski,0.000000,moralno,0.000000,modificirati,0.000000,mladosten,0.000000,milijarden,0.000000
11739,zgostiti,0.000000,gorice,0.000000,nem,0.000000,petindvajset,0.000000,irak,0.000000,...,morebitno,0.000000,moralo,0.000000,modifikacija,0.000000,mladostnik,0.000000,milimeter,0.000000


In [147]:
tf_idf['war']

,SDS,SDS_scores,SD,SD_scores,LDS,LDS_scores,NSi,NSi_scores,SMC,SMC_scores,...,Lipa,Lipa_scores,SLS+SKD,SLS+SKD_scores,ZaAB,ZaAB_scores,Konkretno,Konkretno_scores,DLGV,DLGV_scores
0,ilegalen,0.166588,idrija,0.242179,bručan,0.124626,pomurski,0.202867,smc,0.619666,...,lipa,0.742096,pionir,0.227037,sprava,0.287368,gasilski,0.361885,prezadolženost,0.577288
1,zločin,0.142994,epidemija,0.173603,jerovšek,0.112897,pomurje,0.149952,veteran,0.172565,...,motnja,0.150688,skd,0.227037,vogrin,0.160784,avdiovizualen,0.258704,zadolženost,0.362412
2,klemenčič,0.131834,veteran,0.156135,sok,0.106620,migrantski,0.147402,migrantski,0.167755,...,vdova,0.138737,zagreb,0.209277,pokop,0.143684,gasilstvo,0.230696,zalomiti,0.259080
3,šarec,0.112231,pomurje,0.128981,posrednik,0.103781,sib,0.145271,epidemija,0.137166,...,sod,0.133945,dravski,0.174889,blef,0.116616,boxer,0.161690,vstaviti,0.224320
4,totalitaren,0.111245,idrijski,0.122545,mercator,0.098053,epidemija,0.143816,terorizem,0.088693,...,bahati,0.133351,stečajen,0.174889,žibert,0.112681,romski,0.158325,odhajanje,0.224320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12383,dobrovnik,0.000000,spravljiv,0.000000,poezija,0.000000,manjšinec,0.000000,petrič,0.000000,...,musliman,0.000000,moriti,0.000000,motoriziran,0.000000,moliti,0.000000,mislinja,0.000000
12384,prašen,0.000000,spredaj,0.000000,pogačar,0.000000,manjšanje,0.000000,petrinov,0.000000,...,muslimanski,0.000000,morišče,0.000000,mozaik,0.000000,molk,0.000000,mislinjski,0.000000
12385,ustavnorevizijski,0.000000,komentiranje,0.000000,poginiti,0.000000,manjvreden,0.000000,petnajstkrat,0.000000,...,mussolini,0.000000,mornarica,0.000000,močvirje,0.000000,molotov,0.000000,mit,0.000000
12386,prapor,0.000000,komedija,0.000000,pogledovati,0.000000,rušitev,0.000000,petkov,0.000000,...,muzejski,0.000000,mors,0.000000,moštvo,0.000000,molsti,0.000000,miting,0.000000
